# CSV Profiles Generator for Rocket Nozzle Design

This notebook generates CSV files describing the geometric profiles of a rocket engine nozzle and its wall layers, including cooling channels. It is designed to help you quickly create and export nozzle geometry data for use in mesh generation, simulation, or further CAD processing.

---

## What Does This Notebook Do?

- **Defines nozzle and wall parameters:**  
  Set the main geometric parameters for the nozzle, wall thickness, and cooling channels.

- **Generates nozzle profiles:**  
  Uses Bézier and de Laval curves to create a smooth nozzle shape.

- **Calculates wall and channel layers:**  
  Computes the thickness and width of each layer (inner wall, cooling channel, outer wall) along the nozzle.

- **Exports to CSV:**  
  Saves the main nozzle curve and all layer data to two CSV files for easy use in other tools.

---

## How It Works

1. **Imports and Dependency Checks:**  
   The notebook checks for required Python packages (`numpy`, `matplotlib`, `scipy`, `pandas`, `math`) and installs them if missing.

2. **Parameter Setup:**  
   - Set the start, throat, and end positions and diameters of the nozzle.
   - Define wall thicknesses and cooling channel properties.
   - Choose the number of points for the profile and output CSV filenames.

3. **Profile Generation Functions:**  
   - Functions to create Bézier and de Laval curves for the nozzle profile.
   - Functions to calculate the thickness and width of each wall and channel layer.

4. **Profile and Layer Calculation:**  
   - Generates the main nozzle curve.
   - Calculates the thickness of each wall and channel layer at every point along the nozzle.
   - Computes the width and number of cooling channels.

5. **CSV Export:**  
   - Saves the nozzle curve (`nozzle_curve.csv`) with columns for x and y coordinates.
   - Saves the nozzle layers (`nozzle_layers.csv`) with columns for x, channel height, channel width, hot gas wall thickness, outer wall thickness, and number of channels.

6. **Visualization (Optional):**  
   - If `DEBUG = True`, plots the nozzle profile and wall layers for visual verification.

---

## How to Use

1. **Set your parameters** in the parameter cell (nozzle dimensions, wall thickness, channel properties).
2. **Run all cells** in order.
3. **Find your CSV files** (`nozzle_curve.csv` and `nozzle_layers.csv`) in the working directory.
4. **Use the CSV files** in your mesh generator, CAD tool, or simulation workflow.

---

## Example Output

- `nozzle_curve.csv`:  
  | x      | y      |
  |--------|--------|
  | ...    | ...    |

- `nozzle_layers.csv`:  
  | x      | channel height | channel width | hot gas wall | outer thickness | number channels |
  |--------|---------------|--------------|--------------|-----------------|-----------------|
  | ...    | ...           | ...          | ...          | ...             | ...             |

---

## Notes

- **Parameter validation:**  
  The notebook checks for negative wall thickness and warns you if your parameters are inconsistent.
- **Customization:**  
  You can easily adapt the parameters and functions for different nozzle shapes or channel designs.
- **Visualization:**  
  Enable `DEBUG = True` to see plots of the generated profiles and layers.


In [6]:
import os
import warnings
import time

try:
    import matplotlib.pyplot as plt
except ImportError:
    print("matplotlib is not installed. Trying to install it...")
    os.system("pip install matplotlib")
    import matplotlib.pyplot as plt

try:
    import numpy as np
except ImportError:
    print("numpy is not installed. Trying to install it...")
    os.system("pip install numpy")
    import numpy as np

try: 
    import scipy
    from scipy.optimize import linear_sum_assignment
    from scipy.sparse.csgraph import minimum_spanning_tree
except ImportError:
    print("scipy is not installed. Trying to install it...")
    os.system("pip install scipy")
    import scipy
    from scipy.optimize import linear_sum_assignment
    from scipy.sparse.csgraph import minimum_spanning_tree

try:
    import pandas as pd
except ImportError:
    print("pandas is not installed. Trying to install it...")
    %pip install pandas
    import pandas as pd

try:
    import math
except ImportError:
    print("math is not installed. Trying to install it...")
    %pip install math
    import math
    
try:
    import scipy.interpolate
except ImportError:
    print("scipy.interpolate is not installed. Trying to install it...")
    %pip install scipy
    import scipy.interpolate

In [ ]:
### === Parameters for the nozzle ===

DEBUG = False

# Nozzle parameters
X_Start = 0.0  # Start of the nozzle in the x-direction
X_Throat = 1.0  # Throat of the nozzle in the x-direction
X_End = 3.0    # End of the nozzle in the x-direction
D_Start = 0.3  # Diameter at the start of the nozzle
D_Throat = 0.1  # Diameter at the throat of the nozzle
D_End = 1.0  # Diameter at the end of the nozzle
X_Nozzle_Parameters = [X_Start, X_Throat, X_End]
D_Nozzle_Parameters = [D_Start, D_Throat, D_End]
Nozzle_Wall_Thickness = 0.02  # Thickness of the nozzle wall

# Cooling channel parameters
Gaz_To_Cooling_Channel_Wall_Thickness = 0.005  # Distance from the gas to the cooling channel wall
Gaz_To_Cooling_Channel_Wall_Thickness_Function = lambda x: x*0 + Gaz_To_Cooling_Channel_Wall_Thickness # Function to get the thickness of the gas to cooling channel wall
Cooling_Channel_Thickness = 0.01  # Thickness of the cooling channel wall
Cooling_Channel_Thickness_Function = lambda x: x*0 + Cooling_Channel_Thickness  # Function to get the thickness of the cooling channel wall
Cooling_Channel_Number = 50 # Number of cooling channels
Cooling_Channel_Angle_Size = np.pi / 50 # Angle size of the cooling channel

# Outer wall parameter
Outer_Wall_Thickness = Nozzle_Wall_Thickness - Gaz_To_Cooling_Channel_Wall_Thickness - Cooling_Channel_Thickness  # Thickness of the outer wall
if Outer_Wall_Thickness < 0:
    warnings.warn("Outer wall thickness is negative. Adjust the parameters to ensure a valid design.")
    Outer_Wall_Thickness = 0.001  # Set to a minimum value to avoid negative thickness
Outer_Wall_Thickness_Function = lambda x: x*0 + Outer_Wall_Thickness  # Function to get the thickness of the outer wall

# Parameters of creation of the csv
number_of_points_profile = 200  # Number of points per profile
nozzle_curve_csv_filename = "nozzle_curve.csv" #You can add a folder name before the filename to save it in a specific folder
nozzle_curve_csv_columns = ["x", "y"]
nozzle_layers_csv_filename = "nozzle_layers.csv" #You can add a folder name before the filename to save it in a specific folder
nozzle_lavers_csv_columns = ["x","channel height","channel width","hot gas wall","outer thickness","number channels"]

In [8]:
### === Nozzle geometry functions === ###

def bezier(t, P0, P1, P2, P3):
    """Bézier curve function."""
    return (1 - t)**3 * P0 + 3 * (1 - t)**2 * t * P1 + 3 * (1 - t) * t**2 * P2 + t**3 * P3

def laval(x, XThroat, XExit, P0, P1, P2, P3, P4, P5, P6, P7):
    """Laval nozzle profile function."""
    if x < XThroat:
        t = x / XThroat
        return bezier(t, P0, P1, P2, P3)[1]
    else:
        t = (x - XThroat) / (XExit - XThroat)
        return bezier(t, P4, P5, P6, P7)[1]

def laval_curve_generator(X_vector, D_vector):
    """Generate the de Laval nozzle profile."""
    global number_of_points_profile
    
    XStart, XThroat, XExit = X_vector
    DStart, DThroat, DExit = D_vector
    
    # Bézier control points
    P0 = np.array([XStart, DStart / 2])
    P1 = np.array([XThroat, DStart / 2])
    P2 = np.array([XThroat, DThroat / 2])
    P3 = np.array([XStart, DThroat / 2])
    P4 = np.array([XThroat, DThroat / 2])
    P5 = np.array([XExit, (DThroat + DExit) / 4])
    P6 = np.array([XExit, DExit / 2])
    P7 = np.array([XThroat, DExit / 2])

    # Create the de Laval nozzle profile using Bézier curves
    print("Creating the de Laval nozzle profile...")
    X = np.linspace(XStart, XExit, number_of_points_profile)
    laval_curve = np.array([laval(x, XThroat, XExit, P0, P1, P2, P3, P4, P5, P6, P7) for x in X])
    
    return laval_curve

def create_profile(X_vector, D_vector, name="rocket_engine", debug=False):
    """Generate the rocket engine geometry. """
    
    # Create the de Laval nozzle profile using Bézier curves
    X = np.linspace(X_vector[0], X_vector[2], number_of_points_profile)
    laval_curve = laval_curve_generator(X_vector, D_vector)

    # Convert into a 2D array for plotting
    laval_curve_2D = [[X[i], laval_curve[i]] for i in range(len(laval_curve))]
    laval_curve_2D = np.array(laval_curve_2D)
    
    
    # Create the 2D inner and outer curves from the points
    curve = laval_curve_2D
    
    # Merge the inner and outer curve into one closed curve
    curve = np.array(curve)
    
    # Debug: Plot the nozzle profile
    if debug:
        plt.figure(figsize=(8, 4))
        plt.plot(curve[:, 0], curve[:, 1], label="Inner Curve", color='blue')
        plt.plot(curve[:, 0], curve[:, 1], label="Outer Curve", color='red')
        plt.fill_between(curve[:, 0], curve[:, 1], curve[:, 1], color='gray', alpha=0.5, label="Wall Thickness")
        plt.title(f"Profile of {name}")
        plt.xlabel("X-axis")
        plt.ylabel("Y-axis")
        plt.axhline(0, color='black', lw=0.5, ls='--')
        plt.axvline(0, color='black', lw=0.5, ls='--')
        plt.grid()
        plt.legend()
        plt.axis('equal')
        plt.show()
    
    return curve[:,1]

In [9]:
### === Profiles generator === ###

def generate_profiles(debug=False):
    """Generate the profiles for the rocket engine."""
    
    print("=== STARTING THE GENERATION OF THE PROFILES ===")

    #Get all the parameters
    global X_Nozzle_Parameters, D_Nozzle_Parameters, Nozzle_Wall_Thickness
    global Gaz_To_Cooling_Channel_Wall_Thickness_Function, Cooling_Channel_Thickness_Function, Outer_Wall_Thickness_Function
    global number_of_points_profile, inner_wall_csv_filename, geometry_csv_filename

    # Creating the x coordinates
    x_values = np.linspace(X_Nozzle_Parameters[0], X_Nozzle_Parameters[2], number_of_points_profile)
      
    print("=== GENERATING THE CURVE OF THE NOZZLE ===")
    gaz_wall_thickness = create_profile(X_Nozzle_Parameters, D_Nozzle_Parameters, name="gaz_wall", debug=debug)

    print("=== GENERATING THE FIRST LAYER OF THE NOZZLE WALL ===")
    gaz_to_cooling_channel_thickness = Gaz_To_Cooling_Channel_Wall_Thickness_Function(x_values)
    
    print("=== GENERATING THE THICKNESS OF THE COOLING CHANNEL WALL ===")
    cooling_channel_thickness = Cooling_Channel_Thickness_Function(x_values)
    
    print("=== GENERATING THE OUTER LAYER THICKNESS ===")
    outer_wall_thickness = Outer_Wall_Thickness_Function(x_values)
    
    # Debug: plot the cumulative thicknesses
    if debug:
        inner_nozzle = gaz_wall_thickness
        inner_cooling_channel = gaz_wall_thickness + gaz_to_cooling_channel_thickness
        outer_cooling_channel = inner_cooling_channel + cooling_channel_thickness
        outer_nozzle = outer_cooling_channel + outer_wall_thickness
        plt.figure(figsize=(10, 5))
        plt.plot(x_values, inner_nozzle, label="Inner Nozzle Wall", color='blue')
        plt.plot(x_values, inner_cooling_channel, label="Inner Cooling Channel", color='orange')
        plt.plot(x_values, outer_cooling_channel, label="Outer Cooling Channel", color='green')
        plt.plot(x_values, outer_nozzle, label="Outer Nozzle Wall", color='red')
        plt.title("Cumulative Thicknesses of the Nozzle")
        plt.xlabel("X-axis")
        plt.ylabel("Thickness")
        for i, x in enumerate(X_Nozzle_Parameters):
            plt.axvline(x, color='gray', lw=0.5, ls='--')
            plt.text(x, 0.1, f'X{i}', fontsize=8, verticalalignment='bottom', horizontalalignment='center')
        for i, d in enumerate(D_Nozzle_Parameters):
            plt.axhline(d, color='gray', lw=0.5, ls='--')
            plt.text(0.1, d, f'D{i}', fontsize=8, verticalalignment='center', horizontalalignment='right')
        plt.axhline(0, color='black', lw=0.5, ls='--')
        plt.axvline(0, color='black', lw=0.5, ls='--')
        plt.grid()
        plt.legend()
        plt.axis('equal')
        plt.show()
    
    print("=== GENERATING THE CHANNEL WIDTH ===")
    def angle_to_width(angle, radius):
        """Convert angle to width based on the radius."""
        return 2 * radius * np.sin(angle / 2)
    radius = gaz_to_cooling_channel_thickness + cooling_channel_thickness / 2 + outer_wall_thickness / 2
    cooling_channel_width = [angle_to_width(Cooling_Channel_Angle_Size, r) for r in radius]
    
    print("=== CREATING THE NUMBER OF COOLING CHANNELS ===")
    cooling_channel_number = [int(Cooling_Channel_Number) for x in x_values]
    
    print(f"=== CREATING FIRST FILE : {nozzle_curve_csv_filename} ===")
    # Create the nozzle curve CSV file using the specified columns
    nozzle_curve_data = {
        nozzle_curve_csv_columns[0]: x_values,
        nozzle_curve_csv_columns[1]: gaz_wall_thickness
    }
    nozzle_curve_df = pd.DataFrame(nozzle_curve_data)
    nozzle_curve_df.to_csv(nozzle_curve_csv_filename, index=False)
    
    print(f"=== CREATING SECOND FILE : {nozzle_layers_csv_filename} ===")
    # Create the nozzle layers CSV file using the specified columns
    nozzle_layers_data = {
        nozzle_lavers_csv_columns[0]: x_values,
        nozzle_lavers_csv_columns[1]: cooling_channel_thickness,
        nozzle_lavers_csv_columns[2]: cooling_channel_width,
        nozzle_lavers_csv_columns[3]: gaz_to_cooling_channel_thickness,
        nozzle_lavers_csv_columns[4]: outer_wall_thickness,
        nozzle_lavers_csv_columns[5]: cooling_channel_number
    }
    nozzle_layers_df = pd.DataFrame(nozzle_layers_data)
    nozzle_layers_df.to_csv(nozzle_layers_csv_filename, index=False)
    print("=== PROFILES GENERATED SUCCESSFULLY ===")
    return nozzle_curve_df, nozzle_layers_df


In [10]:
### === Main function to run the script === ###

if __name__ == "__main__":
    start_time = time.time()
    nozzle_curve_df, nozzle_layers_df = generate_profiles(debug=DEBUG)
    end_time = time.time()
    
    print(f"Profiles generated in {end_time - start_time:.2f} seconds.")
    print(f"Nozzle curve saved to {nozzle_curve_csv_filename}")
    print(f"Nozzle layers saved to {nozzle_layers_csv_filename}")
    print("=== END OF THE SCRIPT ===")

=== STARTING THE GENERATION OF THE PROFILES ===
=== GENERATING THE CURVE OF THE NOZZLE ===
Creating the de Laval nozzle profile...
=== GENERATING THE FIRST LAYER OF THE NOZZLE WALL ===
=== GENERATING THE THICKNESS OF THE COOLING CHANNEL WALL ===
=== GENERATING THE OUTER LAYER THICKNESS ===
=== GENERATING THE CHANNEL WIDTH ===
=== CREATING THE NUMBER OF COOLING CHANNELS ===
=== CREATING FIRST FILE : nozzle_curve.csv ===
=== CREATING SECOND FILE : nozzle_layers.csv ===
=== PROFILES GENERATED SUCCESSFULLY ===
Profiles generated in 0.01 seconds.
Nozzle curve saved to nozzle_curve.csv
Nozzle layers saved to nozzle_layers.csv
=== END OF THE SCRIPT ===
